# Cleaning Ebay Car Advertisement Data

In this notebook, we will explore and clean advertisement data, making it more suitable for future analysis. Below we import the data and examine its data dictionary.

## Importing and Exploring Data

In [1]:
import pandas as pd

autos = pd.read_csv('datasets/autos.csv', encoding='Latin-1')
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


At first glance, there appear to be several columns that carry numerical data stored as strings, such as `price` and `odometer`. It also appears that some columns hold time series data stored as strings as well.

Moreover, some data needs to be converted to its English equivalent from German to make it easier to work with. Before examining the data, let's rename some columns. 

Below we list the data dictionary.

* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car.
* `abtest` - Whether the listing is included in an A/B test.
* `vehicleType` - The vehicle Type.
* `yearOfRegistration` - The year in which the car was first registered.
* `gearbox` - The transmission type.
* `powerPS` - The power of the car in PS.
* `model` - The car model name.
* `odometer` - How many kilometers the car has driven.
* `monthOfRegistration` - The month in which the car was first registered.
* `fuelType` - What type of fuel the car uses.
* `brand` - The brand of the car.
* `notRepairedDamage` - If the car has a damage which is not yet repaired.
* `dateCreated` - The date on which the eBay listing was created.
* `nrOfPictures` - The number of pictures in the ad.
* `postalCode` - The postal code for the location of the vehicle.
* `lastSeen` - When the crawler saw this ad last online.

## Renaming Columns

In [2]:
new_col_names = {
    'dateCrawled': 'date_crawled',
    'offerType': 'offer_type',
    'vehicleType': 'vehicle_type',
    'yearOfRegistration': 'registration_year',
    'powerPS': 'power_ps',
    'monthOfRegistration': 'registration_month',
    'fuelType': 'fuel_type',
    'notRepairedDamage': 'damage_not_repaired',
    'dateCreated': 'date_created',
    'nrOfPictures': 'picture_count',
    'postalCode': 'postal_code',
    'lastSeen': 'last_seen',
    'odometer': 'odometer_km'
}
autos.rename(columns=new_col_names, inplace=True)

Most column names were simply changed from camel case to their snake case equivalent. But others were given simpler names, such as `monthOfRegsitration` and `yearOfRegistration`, and given more descriptive names such as `odometer`.

Now let's look for any columns that could serve as the DataFrame index.

## Choosing an Index

In [3]:
uniq_cols = [
    col_name for col_name in autos.columns
    if autos[col_name].is_unique
]

if uniq_cols:
    for c in uniq_cols:
        print(f'\'{c}\' column is unique.')
else:
    print('No unique columns.')

No unique columns.


Because no columns have unique values, we'll keep the index that pandas automatically generated when we imported the data.

## Dropping Columns

In [4]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
date_crawled           50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offer_type             50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicle_type           44905 non-null object
registration_year      50000 non-null int64
gearbox                47320 non-null object
power_ps               50000 non-null int64
model                  47242 non-null object
odometer_km            50000 non-null object
registration_month     50000 non-null int64
fuel_type              45518 non-null object
brand                  50000 non-null object
damage_not_repaired    40171 non-null object
date_created           50000 non-null object
picture_count          50000 non-null int64
postal_code            50000 non-null int64
last_seen              50000 non-null obj

Since the columns with missing data are at least 80% full, we'll keep all of them for now and take a closer look at the data variation in each column before dropping any columns.

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,damage_not_repaired,date_created,picture_count,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 19:48:02,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The `seller` and `offer_type` columns have essentialy no variation; they each hold two unique values, but only hold the same value in every row except one.

In [6]:
autos[['seller', 'offer_type']].describe()

,seller,offer_type
count,50000,50000
unique,2,2
top,privat,Angebot
freq,49999,49999


`seller` holds the value `privat` in 49,999 of 50,000 rows, and `offer_type` holds the value `Angebot` in as many rows. Let's drop these columns since it'll be difficult to find any useful insights by including them in our analysis.

In [7]:
cols_to_drop = ['seller', 'offer_type']
autos.drop(columns=cols_to_drop, axis=1, inplace=True)

`picture_count` also appears to have little variation.

In [8]:
autos['picture_count'].value_counts()

0    50000
Name: picture_count, dtype: int64

In fact, it has none. Let's drop this column.

In [9]:
autos.drop('picture_count', axis=1, inplace=True)

## Tidying Up Fields in Data

#### Cleaning Price and Odomter

In [10]:
cols_to_clean = ['price', 'odometer_km']
autos[cols_to_clean].info()
autos[cols_to_clean].head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
price          50000 non-null object
odometer_km    50000 non-null object
dtypes: object(2)
memory usage: 781.4+ KB


,price,odometer_km
0,"$5,000","150,000km"
1,"$8,500","150,000km"
2,"$8,990","70,000km"
3,"$4,350","70,000km"
4,"$1,350","150,000km"
5,"$7,900","150,000km"
6,$300,"150,000km"
7,"$1,990","150,000km"
8,$250,"150,000km"
9,$590,"150,000km"


Although `price` and `odometer_km` hold strings, they store numerical data. Let's convert these columns to numerical data types.

In [11]:
assert(autos['price'].str.match(r'^\$[\w,]+$')).all()
assert(autos['odometer_km'].str.match(r'^[\w,]+km$')).all()

def remove_col_chars(col, chars):
    cleaned_col = col
    for ch in chars:
        cleaned_col = cleaned_col.str.replace(ch, '')
    return cleaned_col

autos['price'] = remove_col_chars(autos['price'], [',', '$']).astype(int)
autos['odometer_km'] = remove_col_chars(autos['odometer_km'], [',', 'km']).astype(int)

autos[['price', 'odometer_km']].head(10)

,price,odometer_km
0,5000,150000
1,8500,150000
2,8990,70000
3,4350,70000
4,1350,150000
5,7900,150000
6,300,150000
7,1990,150000
8,250,150000
9,590,150000


#### Cleaning Date Data

In [12]:
cols_to_clean = [
    'date_crawled',
    'date_created',
    'last_seen',
    'registration_year',
    'registration_month'
]
autos[cols_to_clean].describe(include='all')

,date_crawled,date_created,last_seen,registration_year,registration_month
count,50000,50000,50000,50000.000000,50000.000000
unique,48213,76,39481,NaN,NaN
top,2016-03-30 19:48:02,2016-04-03 00:00:00,2016-04-07 06:17:27,NaN,NaN
freq,3,1946,8,NaN,NaN
mean,NaN,NaN,NaN,2005.073280,5.723360
std,NaN,NaN,NaN,105.712813,3.711984
min,NaN,NaN,NaN,1000.000000,0.000000
25%,NaN,NaN,NaN,1999.000000,3.000000
50%,NaN,NaN,NaN,2003.000000,6.000000
75%,NaN,NaN,NaN,2008.000000,9.000000


`date_crawled`, `date_created`, and `last_seen` store date as strings conforming to the ISO format `YYYY-MM-DD hh:mm:ss`. So we can quickly convert these column values to datetime objects.

In [13]:
for col in cols_to_clean[:3]:
    assert(autos[col].str.match(r'^\w{4}-\w{2}-\w{2} \w{2}:\w{2}:\w{2}')).all()

for col in cols_to_clean[:3]:
    autos[col] = pd.to_datetime(autos[col])

autos[cols_to_clean].describe(include='all')

,date_crawled,date_created,last_seen,registration_year,registration_month
count,50000,50000,50000,50000.000000,50000.000000
unique,48213,76,39481,NaN,NaN
top,2016-03-12 16:06:22,2016-04-03 00:00:00,2016-04-07 06:17:27,NaN,NaN
freq,3,1946,8,NaN,NaN
first,2016-03-05 14:06:30,2015-06-11 00:00:00,2016-03-05 14:45:46,NaN,NaN
last,2016-04-07 14:36:56,2016-04-07 00:00:00,2016-04-07 14:58:50,NaN,NaN
mean,NaN,NaN,NaN,2005.073280,5.723360
std,NaN,NaN,NaN,105.712813,3.711984
min,NaN,NaN,NaN,1000.000000,0.000000
25%,NaN,NaN,NaN,1999.000000,3.000000


Next we'll convert `registration_year` and `registration_month` into a single string column--from which we'll extract date information--by converting them into string columns and then combining them. But before combining them we explore the data to see if there are any values that might prevent this step from going smoothly.

In [14]:
reg_years = autos['registration_year']
reg_months = autos['registration_month']

workable_year_count = reg_years.between(1900, 2020).sum()
workable_month_count = reg_months.between(1,12).sum()
total_count = autos.shape[0]
workable_year_pct = workable_year_count / total_count * 100
workable_month_pct = workable_month_count / total_count * 100
print(f'workable registration_year values: {workable_year_pct}%')
print(f'workable registration_month values: {workable_month_pct}%')

workable registration_year values: 99.952%
workable registration_month values: 89.85%


Clearly both columns contain nonviable values. `registration_year` contains about 99.9% of its values within the years 1900 to 2020, while 89.9%  of `registration_month`'s values represent actual months. We'll keep all workable values and discard the others, before combining the columns.

In [16]:
import numpy as np

years = autos['registration_year']
months = autos['registration_month']

# keep workable values
years = years.where(years.between(1900, 2020))
months = months.where(months.between(1, 12))

# convert to string
def cast_to_string(series):
    return pd.Series(np.where(series.notnull(), series.astype(str), np.nan))

def pad_with_zeros(series, regex):
    return pd.Series(
        np.where(series.notnull() & series.str.match(regex), '0' + series, series))

years = cast_to_string(years).str[:-2]
months = cast_to_string(months).str[:-2]
months = pad_with_zeros(months, r'^\w$')

# extract date information
autos['registration_date'] = pd.to_datetime(years + '-' + months)
autos[cols_to_clean + ['registration_date']].describe(include='all')

,date_crawled,date_created,last_seen,registration_year,registration_month,registration_date
count,50000,50000,50000,50000.000000,50000.000000,44916
unique,48213,76,39481,NaN,NaN,665
top,2016-03-12 16:06:22,2016-04-03 00:00:00,2016-04-07 06:17:27,NaN,NaN,2003-03-01 00:00:00
freq,3,1946,8,NaN,NaN,376
first,2016-03-05 14:06:30,2015-06-11 00:00:00,2016-03-05 14:45:46,NaN,NaN,1910-01-01 00:00:00
last,2016-04-07 14:36:56,2016-04-07 00:00:00,2016-04-07 14:58:50,NaN,NaN,2019-05-01 00:00:00
mean,NaN,NaN,NaN,2005.073280,5.723360,NaN
std,NaN,NaN,NaN,105.712813,3.711984,NaN
min,NaN,NaN,NaN,1000.000000,0.000000,NaN
25%,NaN,NaN,NaN,1999.000000,3.000000,NaN


The column `registration_date` stores data from `registration_year` and `registration_month` using the datetime data type. Although reducing the number of columns by one, we have consequently introduced more null values into our DataFrame, perhaps motivating us to keep the `registration_year` and `registration_month` columns. But as we mentioned before these columns possess nonsensical values which will distort any analysis that we perform. Thus, dropping these columns would make sense since we extracted all the useful data into `registration_date`.

In [17]:
autos.drop(cols_to_clean[3:], axis=1, inplace=True)